In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

In [ ]:
# Get all letters of the alphabet in a list
chars = sorted(list(set(''.join(words))))

# Create mapping
s_to_i = {s:i for i, s in enumerate(chars, start=1)}
s_to_i['.'] = 0

# Reverse Mapping
i_to_s = {i:s for s, i in s_to_i.items()}
i_to_s

In [199]:
# Build the dataset

def build_dataset(words):
    block_size = 3
    X, Y = [], []

    for w in words:

        context = [0] * block_size
        for ch in w + '.':
            ix = s_to_i[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(i_to_s[i] for i in context), '--->', i_to_s[ix])
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])



torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


In [ ]:
C = torch.randn((27, 2))
C

In [ ]:
# print(C.shape, X.shape, C[X].shape)

# C = (27, 2)    -> Lookup Table: 27 characters, each has a 2D vector
# X = (32, 3)    -> Input Batch: 32 examples, each is a list of 3 indices
# emb = C[X]     -> (32, 3, 2): Every single index in X is replaced by its 2D vector

emb = C[X]

W1 = torch.randn((6, 100))
b1 = torch.randn(100)


### Note to self
We can extend this "chain" in two major directions. In Deep Learning, we actually give these dimensions standard names: B, T, and C.

The current shape is (32, 3, 2). Let's translate that:

* B (Batch) = 32: The stack of parallel examples.

* T (Time) = 3: The length of the chain (sequence length).

* C (Channels) = 2: The depth of information per link (embedding size).

**Extending the Chain (Increasing T)**

Can we chain "a to b to c to d". Yes. This is just increasing the Context Length (or Block Size).

* If we look back 3 characters: (32, 3, 2)

* If we look back 10 characters: (32, 10, 2)

* If we look back 100 characters: (32, 100, 2)

The "chain" of characters gets longer, so the tensor gets wider.

**Extending the Depth (Increasing C)**
   
We can also make the "coordinates" richer. Right now, our characters live in 2D space (2 numbers).

In real models (like GPT), characters live in 768-dimensional space or more.

Shape: (32, 3, 768)

This means every single character is described by a list of 768 numbers, not just 2.

In [ ]:
# Here we want to turn (32, 3, 2) into (32, 6) so we can pass it into the hidden layer (6, 100)
# We can do this using unbind on the emb on the character level dim (3) and concat with coordinates dim (2)
# This results in the flattening of 3x2 into (32,6)

# HOWEVER: this is inefficient because concat creates new memory vs manipulating the tensor view
# --> torch.cat(torch.unbind(emb, 1), 1).shape

# Instead we manipulate the view
# emb.view(-1/emb.shape[0], 6) == torch.cat(torch.unbind(emb, 1), 1)

h_pre = emb.view(-1, 6) @ W1 + b1 # h.shape = (32,100), w1 shape (6, 100), b.shape (100) -> this is broadcasted
h = torch.tanh(h_pre) # hidden layers of activations using tanh

In [ ]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [ ]:
logits = h @ W2 + b2

# Now we want to make logits positive
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)


In [ ]:
# Avg negative log likelihood
loss = -probs[torch.arange(X.shape[0]), Y].log().mean()



## Full Run

In [266]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 5), generator=g)
W1 = torch.randn((15, 200), generator=g)
b1 = torch.randn((200), generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn((27), generator=g)
parameters = [C, W1, b1, W2, b2]

In [267]:
sum(p.nelement() for p in parameters) # number of params in total

8762

In [268]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [269]:
lri = []
lossi = []
stepi = []

for p in parameters:
    p.requires_grad = True

for i in range(50000):

    # mini batch construct
    ix = torch.randint(0, Xtr.shape[0], (32,))

    # forward pass
    emb = C[Xtr[ix]] # (32, 3, 2)
    hidden_pre = emb.view(-1, 15) @ W1 + b1 # (32, 100)
    hidden = torch.tanh(hidden_pre)
    logits = hidden @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    # print(loss.item())

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # learning rate
    lr = 0.01

    # update
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    # lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())
    
# Much more efficient way using cross entropy
# counts = logits.exp()
# probs = counts / counts.sum(1, keepdim = True)
# loss = -probs[torch.arange(X.shape[0]), Y].log().mean()
# loss

In [262]:
print(loss.item())

2.5976619720458984


In [270]:
emb = C[Xtr] # (32, 3, 2)
hidden_pre = emb.view(-1, 15) @ W1 + b1 # (32, 100)
hidden = torch.tanh(hidden_pre)
logits = hidden @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.4012, grad_fn=<NllLossBackward0>)

In [271]:
emb = C[Xdev] # (32, 3, 2)
hidden_pre = emb.view(-1, 15) @ W1 + b1 # (32, 100)
hidden = torch.tanh(hidden_pre)
logits = hidden @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.4146, grad_fn=<NllLossBackward0>)

In [265]:
# plt.plot(lri, lossi)

## Why mini batch

**Why we trade perfect accuracy for speed.**

Mini-batching is one of the most critical practical concepts in deep learning. It’s the difference between waiting **5 days** for a result versus **5 minutes**.

---

### 1. The Problem: "The Classroom Analogy"

Imagine you are a **Teacher** (the Neural Net) trying to improve your **Curriculum** (Weights). You have **30,000 Students** (The Dataset).

#### Option A: Full Batch (Standard Gradient Descent)

* **Method:** You grade the exam of **every single student** (all 30,000). You calculate the average mistake. You make **one** tiny adjustment to your teaching plan.
* **Result:** This is painfully slow. You only improve once per semester.

#### Option B: Stochastic Gradient Descent (SGD)

* **Method:** You grade **one random student**. You change your *entire* curriculum based on that one kid.
* **Result:** **Chaos.** If that kid was a genius or totally lost, you make wild, noisy adjustments. You zigzag everywhere.

---

### 2. The Solution: Mini-Batching (The Sweet Spot)

#### Batch Size: 32

* **Method:** You grab a random group of **32 students**. You grade them and get their average score.
* **The Logic:** The average of 32 random students is usually a pretty good estimate of the average of the whole school. It's not perfect, but it's **"good enough"** to know which direction to move.
* **Result:** You can update your curriculum **1,000 times** in the time it takes the "Full Batch" teacher to update just once.

---

### 3. Why is it "Better"?

It is a trade-off between **Accuracy** and **Speed**, where Speed wins.

### Speed (The Main Reason)

* Calculating gradients for 30,000 rows might take **1 second**.
* Calculating gradients for 32 rows takes **0.001 seconds**.
* **The Benefit:** You can take many more steps down the hill. Even if the steps are slightly "drunk" (approximate), taking **10,000 steps** gets you to the bottom faster than taking 5 perfect steps.

#### Hardware Efficiency

* GPUs love matrices of size **32, 64, 128**. They are physically optimized to process these blocks in parallel.
* Processing **1 item** is inefficient (overhead).
* Processing **1,000,000 items** might crash your RAM.
* **32 is the "Goldilocks" zone.**